In [ ]:
!pip install --upgrade pennylane matplotlib pandas

     |████████████████████████████████| 204kB 2.5MB/s 
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.2.1)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.0.3)


In [ ]:
!pip install pennylane --upgrade

Requirement already up-to-date: pennylane in /usr/local/lib/python3.6/dist-packages (0.8.1)


In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

style.use('bmh')
style.use('dark_background')

## Loading tha manipulation of data

In [ ]:
data = np.asanyarray(pd.read_csv('./LabelledTheta.csv', delimiter=',', header=None))

NUM_CHANNELS = 14
ACTUAL_VALUES = 38
CONST_VALUES = 64 - 38
NUM_THETA_VALUES_PER_CHANNEL = ACTUAL_VALUES + CONST_VALUES
NUM_QUBITS = 6

data

array([['Ankur_sir_6', 0.5116529233737879, 2.661255444814941, ..., 0.0,
        0.0, 'Like'],
       ['Sandeep_32', -0.9532752329297408, -4.419225848654918, ..., 0.0,
        0.0, 'Disike'],
       ['mahendra_28', -0.4930886480644289, -1.5454615508228926, ...,
        0.0, 0.0, 'Like'],
       ...,
       ['mahendra_5', -1.9427615337039161, -12.76711513386849, ..., 0.0,
        0.0, 'Like'],
       ['taufiq_10', -0.34627782376502125, -1.75569285697388, ..., 0.0,
        0.0, 'Disike'],
       ['Vijay_32', 0.42956267802355796, 2.191135723676666, ..., 0.0,
        0.0, 'Disike']], dtype=object)

In [ ]:
def normalize(X_data):
    for i in range(len(X_data)):
        mu = X_data[i].mean()
        sigma = X_data[i].std()

        X_data[i] = (X_data[i] - mu) / sigma

        for j in range(NUM_CHANNELS):
              for k in range(NUM_THETA_VALUES_PER_CHANNEL):
                idx = j * NUM_THETA_VALUES_PER_CHANNEL + k
                if X_data[i][idx] > 2:
                    X_data[i][idx] = 2
                elif X_data[i][idx] < -2:
                    X_data[i][idx] = -2

                if k >= ACTUAL_VALUES:
                    X_data[i][idx] = -2.5

        X_data[i] = ((X_data[i] / 5) + 0.5)
            
    return X_data

In [ ]:
# TODO: change the function for different encoding
def get_angles(X_data):
    X_angles = []
    for i in range(len(X_data)):
        X_i = []
        for j in range(NUM_CHANNELS):
            for q in range(NUM_QUBITS):
                probab_sum = 0.0
                for k in range(NUM_THETA_VALUES_PER_CHANNEL):
                    idx = j * NUM_THETA_VALUES_PER_CHANNEL + k

                    if k & (0x1 << q):
                        probab_sum += X_data[i][idx]

                X_i.append(2 * np.arcsin(np.sqrt(probab_sum)))

        X_angles.append(np.array(X_i))

    return X_angles

In [ ]:
def convert_to_probability(X_data):
    X_data_probab = []
    for i in range(len(X_data)):
        X_i = []
        for j in range(NUM_CHANNELS):
            channel_j = []
            for k in range(NUM_THETA_VALUES_PER_CHANNEL):
                idx = j * NUM_THETA_VALUES_PER_CHANNEL + k
                channel_j.append(X_data[i][idx])

            total_ch_j = sum(channel_j)
            channel_j = np.array(channel_j) / total_ch_j

            for k in range(NUM_THETA_VALUES_PER_CHANNEL):
                X_i.append(channel_j[k])
    
        X_data_probab.append(np.array(X_i))

    return X_data_probab

In [ ]:
def arrange_channelwise(X_data):
    X_data_channelwise = []
    for i in range(NUM_CHANNELS):
        X_i = []
        for j in range(len(X_data)):
            X_j = []
            for k in range(NUM_QUBITS):
                idx = i * NUM_QUBITS + k
                X_j.append(X_data[j][idx])
                
            X_i.append(np.array(X_j))
            
        X_data_channelwise.append(np.array(X_i))
        
    return X_data_channelwise

In [ ]:
X = []
Y = []
for i in range(len(data)):
    X.append(data[i][1:-1])
    Y.append(data[i][-1])

index = np.random.permutation(range(len(X)))
    
X_norm = normalize(X)
X_probab = convert_to_probability(X_norm)
X_ang = np.array(get_angles(X_probab))#[index[:200]]

In [ ]:
print(type(X_ang), type(X_ang[0]))
print(len(X_ang), len(X_ang[0]))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
1045 84


In [ ]:
X_chwise = arrange_channelwise(X_ang)
Y_bin = np.array([1 if y == 'Like' else -1 for y in Y])#[index[:200]]

print(len(X_chwise), len(X_chwise[0]), len(X_chwise[0][0]))

14 1045 6


In [ ]:
train_idxs = int(len(X_ang)*0.7)+1
test_idxs = int(train_idxs + len(X_ang)*0.2)

X_train = X_ang[:train_idxs]
Y_train = Y_bin[:train_idxs]

X_test = X_ang[train_idxs:test_idxs]
Y_test = Y_bin[train_idxs:test_idxs]

X_val = X_ang[test_idxs:]
Y_val = Y_bin[test_idxs:]

In [ ]:
X_train_chwise = arrange_channelwise(X_train)
X_test_chwise = arrange_channelwise(X_test)
X_val_chwise = arrange_channelwise(X_val)

print(len(X_train_chwise), len(X_train_chwise[0]))
print(len(X_test_chwise), len(X_test_chwise[0]))
print(len(X_val_chwise), len(X_val_chwise[0]))

14 732
14 209
14 104


## Creating the quantum network

In [ ]:
def layer(W, num_wires):
    for i in range(num_wires):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
        
    for j in range(num_wires):
        if j != num_wires-1:
            qml.CNOT(wires=[j, j+1])
        else:
            qml.CNOT(wires=[j, 0])

In [ ]:
def statePreparation(a):
    for i in range(len(a)):
        qml.RY(a[i], wires=i)
        qml.Hadamard(wires=i)
        if i == 5:
            qml.CZ(wires=[i, 0])
        else:
            qml.CZ(wires=[i, i+1])

In [ ]:
dev = qml.device('default.qubit', wires=NUM_QUBITS)

@qml.qnode(dev)
def circuit(weights, angles):
    statePreparation(angles)
    
    for W in weights:
        layer(W, NUM_QUBITS)
        
    return qml.expval(qml.PauliZ(0))

## Some helper functions

In [ ]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += ((l - p)**2)
        
    loss /= len(labels)
    return loss

################################################################################################
def accuracy(labels, predictions):
    loss = 0
    pred = [1.0 if p > 0 else -1.0 for p in predictions]
    for l, p in zip(labels, pred):
        if abs(l - p) == 0:
            loss += 1
            
    loss /= len(labels)
    return loss

## Building the variational classifier

In [ ]:
def variational_classifier(var, ang=None):
    weights = var[0]
    bias = var[1]
    return circuit(weights, ang) + bias

In [ ]:
def cost(weights, features, labels):
    predictions = [variational_classifier(weights, f) for f in features]
    return square_loss(labels, predictions)

In [ ]:
def opt_cost(weights, features, labels):
    predictions = [variational_classifier(weights, f) for f in features]
    acc = accuracy(labels, predictions)
    if acc > 0:
      return (square_loss(labels, predictions) / accuracy(labels, predictions))**2
    else:
      return (square_loss(labels, predictions)/0.001)**2

## Fitting the model

In [ ]:
NUM_LAYERS = 10
bias = 0.03
var_init = (np.random.randn(NUM_LAYERS, NUM_QUBITS, 3), bias)
var_init

(array([[[ 0.51403063,  0.50533925, -1.37145404],
         [-0.56094635, -0.12829907, -0.51176513],
         [-0.1623919 , -0.54312889,  0.13685506],
         [-0.6696141 ,  0.07186543, -1.82458106],
         [-1.0645167 , -0.11302851,  3.17768829],
         [ 1.67420435, -0.12362914,  0.8471328 ]],
 
        [[-0.48829406, -0.40047092,  0.47754705],
         [ 1.17958366,  1.62021168, -0.08518219],
         [ 0.86016267,  0.81312794,  0.16885397],
         [ 0.55281688, -0.8384062 , -2.15196279],
         [-0.57030831, -1.38322542,  0.21779753],
         [-0.29144702, -0.89676426, -0.40553241]],
 
        [[-0.3014477 ,  0.09973085,  0.71336771],
         [ 0.48498987,  0.5301705 ,  1.16333818],
         [-0.21370233, -0.19941087,  1.5836834 ],
         [ 1.28266573, -1.14892576,  0.00699279],
         [-0.19859814, -0.24685817,  0.33378595],
         [ 0.62325924,  1.25022594,  1.26557608]],
 
        [[ 0.6814887 ,  0.30763764,  0.04012114],
         [ 0.44793592,  1.45928884, -0.43

In [ ]:
def fit_model(x, y, init_weights, batch_size, epochs, opt, validation_data, cost_calc_data):
    var = init_weights
    history = {'cost':[], 'acc_train':[], 'acc_val':[], 'iter_history':[]}
    num_iter = 50
    
    for e in range(epochs):
        history['iter_history'].append({'cost':[], 'acc_train':[], 'acc_val':[]})
        print("EPOCH: ", e+1)
        for it in range(num_iter):
            # Update the weights by one optimizer step
            batch_index = np.random.randint(0, train_idxs, (batch_size,))
            X_train_batch = x[batch_index]
            Y_train_batch = y[batch_index]
            var = opt.step(lambda v: opt_cost(v, X_train_batch, Y_train_batch), var)

            # Compute predictions on train and validation set
            print('Train iter: ', it+1, end='\t')
            predictions_train = [np.sign(variational_classifier(var, f)) for f in x]
            print('Test iter: ', it+1, end='\t')
            predictions_val = [np.sign(variational_classifier(var, f)) for f in validation_data[0]]
            
            
            # Compute accuracy on train and validation set
            history['iter_history'][e]['acc_train'].append(accuracy(y, predictions_train))
            history['iter_history'][e]['acc_val'].append(accuracy(validation_data[1], predictions_val))
            history['iter_history'][e]['cost'].append(cost(var, cost_calc_data[0], cost_calc_data[1]))
            
            # Print iter-wise value
            print(
                "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
                "".format(it + 1, history['iter_history'][e]['cost'][-1], history['iter_history'][e]['acc_train'][-1],
                history['iter_history'][e]['acc_val'][-1])
            )
        
        # Store parameters for later use
        history['acc_train'].append(sum(history['iter_history'][e]['acc_train'])/num_iter)
        history['acc_val'].append(sum(history['iter_history'][e]['acc_val'])/num_iter)
        history['cost'].append(sum(history['iter_history'][e]['cost'])/num_iter)

        print(
            "\n\nEpoch: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
            "".format(e + 1, history['cost'][-1], history['acc_train'][-1], history['acc_val'][-1])
        )
        
        return history, var

In [ ]:
def evaluate_model(weights, x_test, y_test):
  predictions_test = [np.sign(variational_classifier(weights, f)) for f in x_test]
  results = (cost(weights, x_test, y_test), accuracy(y_test, predictions_test))
  print(
      f'*Evalute*: Cost: {results[0]} | Accuracy: {results[1]}'
  )

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# def export_weights(filename, weights):
#   out_file = open('./drive/My Drive/Quantum Project/'+filename, 'w+')

#   for v in weights[0]:
#     for q in v:
#       for ang in q:
#         out_file.write(f'{ang}, ')
#       out_file.write('\n')
#     out_file.write('\n')

#   out_file.write(f'\n\n {weights[1]}')

#   out_file.close()


# export_weights('init_weights.csv', var_init)

In [ ]:
learn_rate = 0.1
# Channel 3 training
ch3_history, ch3_weights = fit_model(x=X_train_chwise[2],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[2], Y_test),
                         cost_calc_data=(X_chwise[2], Y_bin))

evaluate_model(ch3_weights, X_val_chwise[2], Y_val)
# export_weights('ch3_weights.csv', ch3_weights)

# Channel 4 training
ch4_history, ch4_weights = fit_model(x=X_train_chwise[3],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[3], Y_test),
                         cost_calc_data=(X_chwise[3], Y_bin))

evaluate_model(ch4_weights, X_val_chwise[3], Y_val)
# export_weights('ch4_weights.csv', ch4_weights)

EPOCH:  1
Train iter:  1	Test iter:  1	Iter:     1 | Cost: 1.0413276 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  2	Test iter:  2	Iter:     2 | Cost: 0.9852963 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  3	Test iter:  3	Iter:     3 | Cost: 1.0449466 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  4	Test iter:  4	Iter:     4 | Cost: 1.0269887 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  5	Test iter:  5	Iter:     5 | Cost: 1.0609826 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  6	Test iter:  6	Iter:     6 | Cost: 1.0545937 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  7	Test iter:  7	Iter:     7 | Cost: 1.0022854 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  8	Test iter:  8	Iter:     8 | Cost: 0.9923725 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  9	Test iter:  9	Iter:     9 | Cost: 0.9978495 | Acc train: 0.5642077 | Acc validation: 0.

In [ ]:
# Channel 5 training
ch5_history, ch5_weights = fit_model(x=X_train_chwise[4],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[4], Y_test),
                         cost_calc_data=(X_chwise[4], Y_bin))

evaluate_model(ch5_weights, X_val_chwise[4], Y_val)
# export_weights('ch5_weights.csv', ch5_weights)

# Channel 6 training
ch6_history, ch6_weights = fit_model(x=X_train_chwise[5],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[5], Y_test),
                         cost_calc_data=(X_chwise[5], Y_bin))

evaluate_model(ch6_weights, X_val_chwise[5], Y_val)
# export_weights('ch6_weights.csv', ch6_weights)

EPOCH:  1
Train iter:  1	Test iter:  1	Iter:     1 | Cost: 1.4267515 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  2	Test iter:  2	Iter:     2 | Cost: 1.1246114 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  3	Test iter:  3	Iter:     3 | Cost: 1.0801587 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  4	Test iter:  4	Iter:     4 | Cost: 1.1445822 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  5	Test iter:  5	Iter:     5 | Cost: 1.2658426 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  6	Test iter:  6	Iter:     6 | Cost: 1.3891983 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  7	Test iter:  7	Iter:     7 | Cost: 1.1339594 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  8	Test iter:  8	Iter:     8 | Cost: 0.9961324 | Acc train: 0.5655738 | Acc validation: 0.5550239 
Train iter:  9	Test iter:  9	Train iter:  10	

In [ ]:
learn_rate = 0.1

# Channel 1 training
ch1_history, ch1_weights = fit_model(x=X_train_chwise[0],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[0], Y_test),
                         cost_calc_data=(X_chwise[0], Y_bin))

evaluate_model(ch1_weights, X_val_chwise[0], Y_val)
# export_weights('ch1_weights.csv', ch1_weights)

# Channel 2 training
ch2_history, ch2_weights = fit_model(x=X_train_chwise[1],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[1], Y_test),
                         cost_calc_data=(X_chwise[1], Y_bin))

evaluate_model(ch2_weights, X_val_chwise[1], Y_val)
# export_weights('ch2_weights.csv', ch2_weights)

# Channel 3 training
ch3_history, ch3_weights = fit_model(x=X_train_chwise[2],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[2], Y_test),
                         cost_calc_data=(X_chwise[2], Y_bin))

evaluate_model(ch3_weights, X_val_chwise[2], Y_val)
# export_weights('ch3_weights.csv', ch3_weights)

# Channel 4 training
ch4_history, ch4_weights = fit_model(x=X_train_chwise[3],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[3], Y_test),
                         cost_calc_data=(X_chwise[3], Y_bin))

evaluate_model(ch4_weights, X_val_chwise[3], Y_val)
# export_weights('ch4_weights.csv', ch4_weights)

# Channel 5 training
ch5_history, ch5_weights = fit_model(x=X_train_chwise[4],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[4], Y_test),
                         cost_calc_data=(X_chwise[4], Y_bin))

evaluate_model(ch5_weights, X_val_chwise[4], Y_val)
# export_weights('ch5_weights.csv', ch5_weights)

# Channel 6 training
ch6_history, ch6_weights = fit_model(x=X_train_chwise[5],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[5], Y_test),
                         cost_calc_data=(X_chwise[5], Y_bin))

evaluate_model(ch6_weights, X_val_chwise[5], Y_val)
# export_weights('ch6_weights.csv', ch6_weights)

# Channel 7 training
ch7_history, ch7_weights = fit_model(x=X_train_chwise[6],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[6], Y_test),
                         cost_calc_data=(X_chwise[6], Y_bin))

evaluate_model(ch7_weights, X_val_chwise[6], Y_val)
# export_weights('ch7_weights.csv', ch7_weights)

# Channel 8 training
ch8_history, ch8_weights = fit_model(x=X_train_chwise[7],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[7], Y_test),
                         cost_calc_data=(X_chwise[7], Y_bin))

evaluate_model(ch8_weights, X_val_chwise[7], Y_val)
# export_weights('ch8_weights.csv', ch8_weights)

# Channel 9 training
ch9_history, ch9_weights = fit_model(x=X_train_chwise[8],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[8], Y_test),
                         cost_calc_data=(X_chwise[8], Y_bin))

evaluate_model(ch9_weights, X_val_chwise[8], Y_val)
# export_weights('ch9_weights.csv', ch9_weights)

# Channel 10 training
ch10_history, ch10_weights = fit_model(x=X_train_chwise[9],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[9], Y_test),
                         cost_calc_data=(X_chwise[9], Y_bin))

evaluate_model(ch10_weights, X_val_chwise[9], Y_val)
# export_weights('ch10_weights.csv', ch10_weights)

# Channel 11 training
ch11_history, ch11_weights = fit_model(x=X_train_chwise[10],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[10], Y_test),
                         cost_calc_data=(X_chwise[10], Y_bin))

evaluate_model(ch11_weights, X_val_chwise[10], Y_val)
# export_weights('ch11_weights.csv', ch11_weights)

# Channel 12 training
ch12_history, ch12_weights = fit_model(x=X_train_chwise[11],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[11], Y_test),
                         cost_calc_data=(X_chwise[11], Y_bin))

evaluate_model(ch12_weights, X_val_chwise[11], Y_val)
# export_weights('ch12_weights.csv', ch12_weights)

# Channel 13 training
ch13_history, ch13_weights = fit_model(x=X_train_chwise[12],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[12], Y_test),
                         cost_calc_data=(X_chwise[12], Y_bin))

evaluate_model(ch13_weights, X_val_chwise[12], Y_val)
# export_weights('ch13_weights.csv', ch13_weights)

# Channel 14 training
ch14_history, ch14_weights = fit_model(x=X_train_chwise[13],
                         y=Y_train,
                         init_weights=var_init,
                         batch_size=2,
                         epochs=1,
                         opt=AdamOptimizer(learn_rate),
                         validation_data=(X_test_chwise[13], Y_test),
                         cost_calc_data=(X_chwise[13], Y_bin))

evaluate_model(ch14_weights, X_val_chwise[13], Y_val)
# export_weights('ch14_weights.csv', ch14_weights)


# Plotting it
plt.plot(ch1_history['acc_val'], label='Ch-1')
plt.plot(ch2_history['acc_val'], label='Ch-2')
plt.plot(ch3_history['acc_val'], label='Ch-3')
plt.plot(ch4_history['acc_val'], label='Ch-4')
plt.plot(ch5_history['acc_val'], label='Ch-5')
plt.plot(ch6_history['acc_val'], label='Ch-6')
plt.plot(ch7_history['acc_val'], label='Ch-7')
plt.plot(ch8_history['acc_val'], label='Ch-8')
plt.plot(ch9_history['acc_val'], label='Ch-9')
plt.plot(ch10_history['acc_val'], label='Ch-10')
plt.plot(ch11_history['acc_val'], label='Ch-11')
plt.plot(ch12_history['acc_val'], label='Ch-12')
plt.plot(ch13_history['acc_val'], label='Ch-13')
plt.plot(ch14_history['acc_val'], label='Ch-14')
plt.title('Hybrid QCNN performance')
plt.xlabel('Epochs')
plt.legend()
plt.ylabel('Validation Accuracy')
plt.show()

EPOCH:  1
Train iter:  1	Test iter:  1	Iter:     1 | Cost: 1.5043631 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  2	Test iter:  2	Iter:     2 | Cost: 1.0280816 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  3	Test iter:  3	Iter:     3 | Cost: 1.1120511 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  4	Test iter:  4	Iter:     4 | Cost: 1.0295712 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  5	Test iter:  5	Iter:     5 | Cost: 0.9890558 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  6	Test iter:  6	Iter:     6 | Cost: 1.2459408 | Acc train: 0.4357923 | Acc validation: 0.4449761 
Train iter:  7	Test iter:  7	Iter:     7 | Cost: 1.0098268 | Acc train: 0.4385246 | Acc validation: 0.4593301 
Train iter:  8	Test iter:  8	Iter:     8 | Cost: 0.9945529 | Acc train: 0.5642077 | Acc validation: 0.5550239 
Train iter:  9	Test iter:  9	Iter:     9 | Cost: 1.0071522 | Acc train: 0.5642077 | Acc validation: 0.